In [1]:
# script for adapting marginal grid emissions to average grid emissions

# Plan

Need to make new emissions file with additional Amtrak demand

Scale down linearly to eliminate the percentage associated with Amtrak to define a new baseline

Difference between base+Amtrak and scaled base+ Amtrak is the avg. attributable to Amtrak

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import xarray as xr 
import dask

In [2]:
overwriteMarginal = True
overwriteAverage = False

In [63]:
case_name = 'amtk_current'
base_load = pd.read_csv('~/us_ego/inputs/inputs_load.csv') # MWh
amtk_load = pd.read_csv(f'~/us_ego/inputs/inputs_load_{case_name}.csv') # MWh

del_load = amtk_load['demandLoad'] - base_load['demandLoad'] # load attributable to Amtrak

mask = del_load > 0
del_load = del_load[mask]
range_idx = del_load.index
print(del_load)

        Unnamed: 0         r     t  demandLoad
0                0  ERC_REST     1     29087.0
1                1  ERC_REST     2     29081.0
2                2  ERC_REST     3     29428.0
3                3  ERC_REST     4     30073.0
4                4  ERC_REST     5     31131.0
...            ...       ...   ...         ...
534294      534294   WECC_WY  8755      1431.0
534295      534295   WECC_WY  8756      1405.0
534296      534296   WECC_WY  8757      1355.0
534297      534297   WECC_WY  8758      1283.0
534298      534298   WECC_WY  8759      1270.0

[534299 rows x 4 columns]


In [46]:
# open existing file; amtrak load + base load grid emissions
path='~/us_ego/annual_emissions/'
name =f'inventory_power_plants_{case_name}.nc'
base_name = f'inventory_power_plants_base.nc'
amtrak_grid_emissions = xr.open_dataset(path+name, engine="netcdf4")
base_grid_emissions = xr.open_dataset(path+base_name, engine="netcdf4")

# Using Dask for large arrays
amtrak_grid_chunked = amtrak_grid_emissions.chunk({"time": 100})  
base_grid_chunked = base_grid_emissions.chunk({"time": 100})  

amtrak_grid = amtrak_grid_chunked.fillna(0)
base_grid = base_grid_chunked.fillna(0)

In [61]:
print('Base NOx emissions:',(np.sum(base_grid["NO"].values)+np.sum(base_grid["NO2"].values))*3600*123210000/1000/1000,'Tg/yr') # Tg/yr

Base NOx emissions: 3291.371893806383 Tg/yr


In [62]:
print('Amtrak+base NOx emissions:',(np.sum(amtrak_grid['NO'].values)+np.sum(amtrak_grid['NO2'].values))*3600*123210000/1000/1000,'Tg/yr') # Tg/yr

Amtrak+base NOx emissions: 3290.2476592585444 Tg/yr


In [60]:
# initialize array of hourly scaling values
phi_array = np.zeros(len(range_idx))
init_idx = int(range_idx[0])
jj = 0

for ii in range_idx:
    #print(amtk_load['r'][ii],amtk_load['t'][ii],amtk_load['demandLoad'][ii])
    phi_array[jj] = del_load[ii]/(amtk_load['demandLoad'][ii])
    jj += 1
    
print('Maximum fraction of grid load:',round(np.max(phi_array)*100, 2),'%')

Maximum fraction of grid load: 0.27 %


In [19]:
#print(base_load['r'].unique())
region='ERC_rest'
tt = 5
t_low = tt
t_high = tt + 2
'@t_low < t < @t_high'
amtk_load_slice = amtk_load[(amtk_load['t'] < t_high) & (amtk_load['r']==region)]
print(amtk_load_slice)

Empty DataFrame
Columns: [Unnamed: 0, r, t, demandLoad]
Index: []


In [5]:
t = 1
t_high = t + 1
t_low = t - 1

dat1 = base_load.query('@t_low < t < @t_high')
print(dat1)

               r  t  demandLoad
0       ERC_REST  1       29087
8759    ERC_WEST  1        2655
17518       FRCC  1       17709
26277   MAP_WAUE  1        2551
35036     MIS_IA  1        2332
...          ... ..         ...
490504  WECC_SCE  1       10605
499263   WECC_SF  1         542
508022  WECC_SNV  1        2196
516781   WECC_UT  1        4050
525540   WECC_WY  1        1356

[61 rows x 3 columns]


In [6]:
# open existing file; amtrak load + base load grid emissions
path='~/us_ego/annual_emissions/'
name =f'inventory_power_plants_{case_name}.nc'
base_name = f'inventory_power_plants_base.nc'
amtrak_grid_emissions = xr.open_dataset(path+name, engine="netcdf4")
base_grid_emissions = xr.open_dataset(path+base_name, engine="netcdf4")

In [23]:
#print(amtrak_grid_emissions)
from datetime import datetime, timedelta
time = datetime(2016,1,1,0,0,0)
print(time)
dt = timedelta(hours=1)
print(dt)
print(time + dt)

2016-01-01 00:00:00
1:00:00
2016-01-01 01:00:00


In [21]:
#species_list = [i for i in amtrak_grid_emissions.data_vars] 
#print(species_list)

In [20]:
# Using Dask for large arrays
amtrak_grid = amtrak_grid_emissions.chunk({"time": 100})  
base_grid = base_grid_emissions.chunk({"time": 100})  

print(amtrak_grid)

<xarray.Dataset> Size: 38GB
Dimensions:  (time: 8760, lat: 400, lon: 900)
Coordinates:
  * time     (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-30T23:00:00
  * lat      (lat) float64 3kB 20.05 20.15 20.25 20.35 ... 59.75 59.85 59.95
  * lon      (lon) float64 7kB -139.9 -139.8 -139.8 ... -50.25 -50.15 -50.05
Data variables:
    NO       (time, lat, lon) float32 13GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>
    NO2      (time, lat, lon) float32 13GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>
    SO2      (time, lat, lon) float32 13GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>
Attributes:
    Title:        Hourly CH power plant inventory
    Conventions:  COARDS
    History:      Created 2025-04-26
    Contact:      aswhite@mit.edu


In [23]:
if overwriteMarginal == True:
    # Marginal Amtrak emissions: calculate difference between amtk_grid and base_grid
    marginal_diff_grid = xr.ufuncs.subtract(amtrak_grid, base_grid) # ufuncs automatically uses dask chunks

In [24]:
if overwriteMarginal == True:
    print(marginal_diff_grid)

<xarray.Dataset> Size: 76GB
Dimensions:  (time: 8760, lat: 400, lon: 900)
Coordinates:
  * time     (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-30T23:00:00
  * lat      (lat) float64 3kB 20.05 20.15 20.25 20.35 ... 59.75 59.85 59.95
  * lon      (lon) float64 7kB -139.9 -139.8 -139.8 ... -50.25 -50.15 -50.05
Data variables:
    NO       (time, lat, lon) float64 25GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>
    NO2      (time, lat, lon) float64 25GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>
    SO2      (time, lat, lon) float64 25GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>


In [25]:
if overwriteMarginal == True:
    # save to Dataset to path
    path='~/fs11/amtrak_emissions'
    name = (f'inventory_power_plants_{case_name}_marginal.nc')
    marginal_diff_grid.to_netcdf(path+name)

In [26]:
print(marginal_diff_grid)

<xarray.Dataset> Size: 76GB
Dimensions:  (time: 8760, lat: 400, lon: 900)
Coordinates:
  * time     (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-30T23:00:00
  * lat      (lat) float64 3kB 20.05 20.15 20.25 20.35 ... 59.75 59.85 59.95
  * lon      (lon) float64 7kB -139.9 -139.8 -139.8 ... -50.25 -50.15 -50.05
Data variables:
    NO       (time, lat, lon) float64 25GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>
    NO2      (time, lat, lon) float64 25GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>
    SO2      (time, lat, lon) float64 25GB dask.array<chunksize=(100, 400, 900), meta=np.ndarray>


In [5]:
# open existing file; amtrak load + base load grid emissions
path='~/fs11/amtrak_emissions/'
name_1 = (f'inventory_power_plants_{case_name}_marginal.nc')
name_2 = (f'inventory_power_plants_{case_name}_average.nc')
marginal_diff_grid = xr.open_dataset(path+name_1, engine="netcdf4")
average_diff_grid = xr.open_dataset(path+name_2, engine="netcdf4")

In [6]:
marginal_diff_grid = marginal_diff_grid.fillna(0) # kg/m2/s
average_diff_grid = average_diff_grid.fillna(0) # kg/m2/s

MemoryError: Unable to allocate 23.5 GiB for an array with shape (8760, 400, 900) and data type float64

In [7]:
print('Annual grid average NOx emissions from Amtrak:',(np.sum(marginal_diff_grid["NO"].values)+np.sum(marginal_diff_grid["NO2"].values))*3600*123210000/1000,'Mg/yr') # Mg/yr
print('Annual grid average NOx emissions from Amtrak:',(np.sum(average_diff_grid["NO"].values)+np.sum(average_diff_grid["NO2"].values))*3600*123210000/1000,'Mg/yr') # Mg/yr

MemoryError: Unable to allocate 23.5 GiB for an array with shape (8760, 400, 900) and data type float64